In [1]:
import pandas as pd

from utils.assigner import RandomAssigner, GreedyAssigner, OptimizationAssigner
from utils.evaluation import (
    calc_pair_frequencies,
    count_pair_frequency_distribution,
)

In [2]:
total_participants = 15
total_rooms = 3
total_rounds = 3

random_assigner = RandomAssigner(total_participants, total_rooms, total_rounds)
greedy_assigner = GreedyAssigner(total_participants, total_rooms, total_rounds)
optimization_assigner = OptimizationAssigner(
    total_participants, total_rooms, total_rounds
)

random_assignments = random_assigner.generate_assignments()
greedy_assignments = greedy_assigner.generate_assignments()
optimization_assignments = optimization_assigner.generate_assignments()

print(random_assignments)
print(greedy_assignments)
print(optimization_assignments)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/apple/Desktop/Github/room-assignment-api/.venv/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/_g/th1k5jkx66zf5kvxqw66pb6c0000gn/T/88631a5da4c041c7bd916ec355da4152-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/_g/th1k5jkx66zf5kvxqw66pb6c0000gn/T/88631a5da4c041c7bd916ec355da4152-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 971 COLUMNS
At line 4074 RHS
At line 5041 BOUNDS
At line 5402 ENDATA
Problem MODEL has 966 rows, 360 columns and 2340 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 0 - 0.00 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 315 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 315 strengthened rows, 0 substitutions
Cgl0004I processed model has 963 rows, 360 columns (360 integer (360 of w

In [19]:
from tqdm import tqdm

res = []
for _ in tqdm(range(100)):
    random_assigner = RandomAssigner(total_participants, total_rooms, total_rounds)
    greedy_assigner = GreedyAssigner(total_participants, total_rooms, total_rounds)
    optimization_assigner = OptimizationAssigner(
        total_participants, total_rooms, total_rounds
    )

    random_assignments = random_assigner.generate_assignments()
    greedy_assignments = greedy_assigner.generate_assignments()
    optimization_assignments = optimization_assigner.generate_assignments()

    random_pair_freq = calc_pair_frequencies(random_assignments)
    greedy_pair_freq = calc_pair_frequencies(greedy_assignments)
    optimization_pair_freq = calc_pair_frequencies(optimization_assignments)

    random_freq_dist = count_pair_frequency_distribution(random_pair_freq)
    greedy_freq_dist = count_pair_frequency_distribution(greedy_pair_freq)
    optimization_freq_dist = count_pair_frequency_distribution(optimization_pair_freq)

    onepair_list = [
        random_freq_dist.get(1),
        greedy_freq_dist.get(1),
        optimization_freq_dist.get(1),
    ]
    res.append(onepair_list)

In [15]:
onepair_df = pd.DataFrame(res, columns=["random", "greedy", "optimization"])

onepair_df["optimization_best"] = (
    (onepair_df["optimization"] >= onepair_df["random"])
    & (onepair_df["optimization"] >= onepair_df["greedy"])
).astype(int)

onepair_df["greedy_best"] = (
    (onepair_df["greedy"] >= onepair_df["random"])
    & (onepair_df["greedy"] >= onepair_df["optimization"])
).astype(int)

onepair_df["random_best"] = (
    (onepair_df["random"] >= onepair_df["greedy"])
    & (onepair_df["random"] >= onepair_df["optimization"])
).astype(int)

onepair_df

,random,greedy,optimization,optimization_best,greedy_best,random_best
0,40,55,55,1,1,0
1,39,54,64,1,0,0
2,43,55,64,1,0,0
3,44,52,66,1,0,0
4,40,52,60,1,0,0
...,...,...,...,...,...,...
95,43,54,57,1,0,0
96,44,57,61,1,0,0
97,49,52,66,1,0,0
98,44,57,57,1,1,0


In [17]:
(
    onepair_df.optimization_best.sum(),
    onepair_df.greedy_best.sum(),
    onepair_df.random_best.sum(),
)

(np.int64(85), np.int64(23), np.int64(2))

15名, 3ルーム, 3ラウンドでの勝率比較（同率1位あり）：
- 組み合わせ最適化：85%
- Greedy：23%
- ランダム：2%

